In [1]:
import os                                                                                                #整理模块
import pandas as pd
import numpy as np                                                                                       #向量化数组模块，加快运算
import datetime as dt                                                                                    #顾名思义，时间模块
import xarray as xr                                                                                      #三维数据组
import netCDF4                                                                                           #针对气象数据的数据格式
from netCDF4 import Dataset, num2date
import cartopy                                                                                           #地理绘图软件
import cartopy.crs as ccrs                                                                               #坐标参考系统
import cartopy.feature as cfeature                                                                       #地理特征
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER                                #经纬度
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))
import cmocean                                                                                            #色条
from matplotlib import pyplot as plt
# %config InlineBackend.figure_format = 'svg'
%matplotlib inline

/usr/lib/gdalplugins/ogr_Parquet.so: undefined symbol: _ZNK5arrow8DataType18ComputeFingerprintB5cxx11Ev
/usr/lib/gdalplugins/ogr_Parquet.so: undefined symbol: _ZNK5arrow8DataType18ComputeFingerprintB5cxx11Ev
libpodofo.so.0.9.8: cannot open shared object file: No such file or directory
libpodofo.so.0.9.8: cannot open shared object file: No such file or directory
/usr/lib/gdalplugins/ogr_Arrow.so: undefined symbol: _ZNK5arrow8DataType18ComputeFingerprintB5cxx11Ev
/usr/lib/gdalplugins/ogr_Arrow.so: undefined symbol: _ZNK5arrow8DataType18ComputeFingerprintB5cxx11Ev
/usr/lib/gdalplugins/ogr_Parquet.so: undefined symbol: _ZNK5arrow8DataType18ComputeFingerprintB5cxx11Ev
/usr/lib/gdalplugins/ogr_Parquet.so: undefined symbol: _ZNK5arrow8DataType18ComputeFingerprintB5cxx11Ev
libpodofo.so.0.9.8: cannot open shared object file: No such file or directory
libpodofo.so.0.9.8: cannot open shared object file: No such file or directory
/usr/lib/gdalplugins/ogr_Arrow.so: undefined symbol: _ZNK5arrow8Data

In [2]:
def geodetector(vari_x, vari_y):     
    import numpy as np
    SST = np.var (vari_y)
    SSW = np.sum (vari_x)
    q = 1-SSW/SST
    if q > 1:
        print("Please check your data")
        return q
       
    if q >= 0.6 and q <= 1:
        print("the the layout of former factor decided by the latter factors")
        return q
    if q >= 0.3 and q < 0.6:
        print("the the layout of former factor slightly decided by the latter factors")
        return q
    if q < 0.3 and q >= 0:
        print("there are no significant relationship between these two factors")
        return q
    if q <  0:
        print("there is absolutely no relationship between them, but it is better to check your data input")
        return q



In [3]:
def find_nearest(array, value):
    '''
    Find index of nearest value in a numpy array
    '''
    
    array = np.asarray(array)
    idx = (np.absolute(array - value)).argmin()
    
    return idx

4.coral coverage and pH

In [4]:
for item in range(6):
    year = "201" + str(item+1)
    locals()['PH' + str(item+1)] = xr.open_dataset('phchange03'+ year +'.nc')

In [5]:
selectedVariable = 'PH' 
selectedTimeIndex = 0       
selectedDepthIndex = -1     

In [6]:
phmeanmarch = (PH1[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +PH2[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +PH3[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +PH4[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +PH5[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +PH6[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:])/6

In [7]:
ph1603relative = PH6[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:] - phmeanmarch

In [8]:
selectedDepthIndex = -1
bc = pd.read_csv('suitable coral coverage.csv')

In [9]:
pHchange = []
i = 0
for i in range(8):
    site_lat = bc.latitude[int(i)] 
    site_lon = bc.longitude[int(i)]
    lat = ph1603relative['latitude'][:]
    lon = ph1603relative['longitude'][:]
    selectedLatIndex = find_nearest(lat,site_lat)
    sLat = lat[selectedLatIndex]
    selectedLonIndex = find_nearest(lon,site_lon)
    sLon = lon[selectedLonIndex]
    pH = float(ph1603relative.sel(latitude = sLat, longitude = sLon))
    pHchange.append(pH)



In [10]:
bc['pH_change'] = pHchange
# BC = pd.DataFrame(pHchange)
# BC.to_csv('bleaching change.csv', mode = 'a', index = None)

In [14]:
gb = bc.groupby('coveragechange')
a = gb.pH_change.var()
b = bc.pH_change
geodetector (a, b)

there is absolutely no relationship between them, but it is better to check your data input


-2.5733654791422382

In [15]:
#这是正确的思路

gb = bc.groupby('pH_change')
a = gb.coveragechange.var()
b = bc.coveragechange
geodetector (a, b)

the the layout of former factor decided by the latter factors


0.7880794701986755

5.coral coverage and temp

In [11]:
for item in range(6):
    year = "201" + str(item+1)
    locals()['TMP' + str(item+1)] = xr.open_dataset('tmpchange03'+ year +'.nc')
    
selectedVariable = 'temp' 
selectedTimeIndex = 0       
selectedDepthIndex = -1     

In [12]:
tmpmeanmarch = (TMP1[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +TMP2[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +TMP3[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +TMP4[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +TMP5[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +TMP6[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:])/6

tmp1603relative = TMP6[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:] - tmpmeanmarch

In [13]:
tempchange = []
i = 0
for i in range(8):
    site_lat = bc.latitude[int(i)] 
    site_lon = bc.longitude[int(i)]
    lat = tmp1603relative['latitude'][:]
    lon = tmp1603relative['longitude'][:]
    selectedLatIndex = find_nearest(lat,site_lat)
    sLat = lat[selectedLatIndex]
    selectedLonIndex = find_nearest(lon,site_lon)
    sLon = lon[selectedLonIndex]
    Tmp = float(tmp1603relative.sel(latitude = sLat, longitude = sLon))
    tempchange.append(Tmp)

In [14]:
bc['temp_change'] = tempchange

In [15]:
gb2 = bc.groupby('coveragechange')
a2 = gb2.temp_change.var()
b2 = bc.temp_change
geodetector (a2, b2)

there is absolutely no relationship between them, but it is better to check your data input


-2.778499045299532

In [16]:
gb2 = bc.groupby('temp_change')
a2 = gb2.coveragechange.var()
b2 = bc.coveragechange
geodetector (a2, b2)

the the layout of former factor decided by the latter factors


0.7880794701986755

6.coral coverage and salinity

In [17]:
for item in range(6):
    year = "201" + str(item+1)
    locals()['Salinity' + str(item+1)] = xr.open_dataset('saltchange03'+ year +'.nc')

selectedVariable = 'salt' 
selectedTimeIndex = 0       
selectedDepthIndex = -1     

In [18]:
saltmeanmarch = (Salinity1[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +Salinity2[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +Salinity3[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +Salinity4[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +Salinity5[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:]
                +Salinity6[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:])/6

salt1603relative = Salinity6[selectedVariable][selectedTimeIndex, selectedDepthIndex, :,:] - saltmeanmarch

In [19]:
saltchange = []
i = 0
for i in range(8):
    site_lat = bc.latitude[int(i)] 
    site_lon = bc.longitude[int(i)]
    lat = salt1603relative['latitude'][:]
    lon = salt1603relative['longitude'][:]
    selectedLatIndex = find_nearest(lat,site_lat)
    sLat = lat[selectedLatIndex]
    selectedLonIndex = find_nearest(lon,site_lon)
    sLon = lon[selectedLonIndex]
    Salt = float(salt1603relative.sel(latitude = sLat, longitude = sLon))
    saltchange.append(Salt)

In [20]:
bc['salt_change'] = saltchange

In [30]:
gb3 = bc3.groupby('coveragechange')
a3 = gb3.salt_change.var()
b3 = bc3.salt_change
geodetector (a3, b3)

the the layout of former factor slightly decided by the latter factors


0.48133069056210165

In [32]:
gb3 = bc3.groupby('salt_change')
a3 = gb3.coveragechange.var()
b3 = bc3.coveragechange
geodetector (a3, b3)

the the layout of former factor decided by the latter factors


0.7880794701986755

In [22]:
bc.to_csv('coveragegeo.csv')